# 시계열 데이터 예측 작업을 위한 모델 선택 방법

## 시계열 데이터란?
시계열 데이터 예측 모델에 앞서 시계열 데이터란 무엇인가 간단하게 짚어 보겠습니다.

### 시계열 데이터
시계열 데이터는 **하나의 변수가 시간에 따라 측정되는 특별한 유형의 데이터**입니다. 특별한 이유는 간단합니다. 웹사이트를 방문한 사람들의 데이터는 각각 독립적입니다. 그러나 애플의 주식을 생각해보죠 1시간 단위의 가격은 각각 독립적이라 할 수 있을까요? 그렇지 못합니다. 즉 시계열 데이터는 **다른 데이터 요소 간에 관계가 있는 데이터**라는 것입니다.

### 시계열 데이터 특징
이런 특별한 시계열 데이터는 크게 두 가지로 분류가 가능합니다.
일변량 시계열 모델과 다변량 시계열 모델입니다.
일변량 시계열 모델은 미래를 예측하기 위해 하나의 변수와그 시간적 변동만을 사용하는 예측 모델이고 다변량 시계열 모델은 외부 변수를  일변량 시계열 모델에 통합하여 적용한 모델입니다. 특징은 아래와 같습니다.


|일변량 시계열 모델|다변량 시계열 모델|
|-----|-----|
|하나의 변수 사용|여러 변수 사용|
|외부 데이터를 사용할 수 없음|외부 데이터 사용 가능|
|오직 과거와 현재의 관계에 기초|과거와 현재, 변수간의 관계를 바탕으로 함|


### 시계열 분해
시계열 분해는 데이터에서 여러 유형의 변수를 추출하는 기술입니다. 시간 데이터에는 계절성, 추세, 잡음(노이즈)라는 3가지 핵심 요소가 존재합니다.

- **계절성(Seasonality)**은 시계열 데이터에서 나타나는 반복적인 움직임을 이야기합니다. 여름에는 기온이 높고 겨울에는 기온이 낮습니다. 이게 1년마다 반복됩니다. 1년의 데이터 흐름은 매년 비슷하겠죠? 이것이 계절성입니다.
- **추세(Trend)**는 장기적인 상승 혹은 하락 패턴을 이야기합니다. 지구 온난화로인한 기온의 변화에 따라 평균 온도가 상승한다면 지구의 온도는 상승하는 추세를 갖겠죠?
- **잡음(noise)**는 계절성이나 추세로는 설명할 수 없는 시계열의 변동성의 일부를 뜻합니다. 4월에 느닷없이 눈이 내렸습니다. 분명 이상기후 현상이고 계절성과 추세로는 설명하기 어려운 부분입니다.

Co2 데이터 셋을 활용해 간단한 시계열 분해 작업을 진행해봅시다.

~~~
import statsmodels.datasets.co2 as co2

co2_data = co2.load(as_pandas=True).data
co2_data.plot()
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/How-to-Select-a-Model-For-Your-Time-Series-Prediction-Task_10.png?resize=576%2C386&ssl=1)

이제 시계열 분해를 해볼건데 모듈의 seasons_decompose()를 활용하면 쉽게 분해할 수 있습니다.

~~~
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(co2_data)
result.plot()
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/How-to-Select-a-Model-For-Your-Time-Series-Prediction-Task_25.png?resize=593%2C390&ssl=1)

이를 통한 결과는 Co2 데이터는 상승 추세가 있고 강한 계절성을 나타내고 있다는 것입니다.


### 자기상관(Autocorrelation)
자기상관이란 현재 값과 거거 값 간의 상관관계를 이야기합니다. 이는 시계열 데이테가 보여주는 주요 특성중 하나입니다. 자기 상관계수는 양수일수도 음수일수도 있습니다. 각각의 특징은 다음과 같습니다.
- **양의 상관**은 현재 높은 가치가 미래에 높은 가치를 만들 가능성을 의미합니다. 반대의 경우도 성립합니다. 간단하게 주식시장을 생각하면 됩니다.
- **음의 상관**은 반대입니다. 오늘의 높은 값은 내일의 낮은 값을, 오늘의 낮은 값은 내일의 높은 값을 의미합니다. 

이러한 자기 상관은 ACF, PACF plot을 활용해 확인할 수 있습니다.

#### ACF

~~~
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(co2_data)
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/How-to-Select-a-Model-For-Your-Time-Series-Prediction-Task_22.png?resize=580%2C412&ssl=1)

x축은 시간의 역흐름 y축은 '현재 시간'과 모든 시간 단계와의 상관을 확인할 수 있습니다. 해당 그래프에서는 중요한 자기상관이 있다는 것이 확실해 보입니다.


#### PACF
ACF의 한계를 극복하기 위한 대안이 PACF입니다. ACF와 다른점은 모든 시간이 아닌 특정 시점을 지정한 상관 분석이라는 점입니다. 현재를 기준으로한 상관이 아닌 특정한 두 시점의 상관을 분석하는데 의미가 있습니다.

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/How-to-Select-a-Model-For-Your-Time-Series-Prediction-Task_14.png?resize=573%2C406&ssl=1)


### 정상성(Stationarity)
모든 시계열이 추세를 갖지 않습니다. 고정 시계열이 대표적입니다. 이렇게 시간의 흐름에 따라 평균과 분산이 일정한 것을 정상성(Stationarity)라고 합니다.

이 역시 간단한 코드를 통해 확인히 가능합니다.

~~~
from statsmodels.tsa.stattools import adfuller
adf, pval, usedlag, nobs, crit_vals, icbest =  adfuller(co2_data.co2.values)
print('ADF test statistic:', adf)
print('ADF p-values:', pval)
print('ADF number of lags used:', usedlag)
print('ADF number of observations:', nobs)
print('ADF critical values:', crit_vals)
print('ADF best information criterion:', icbest)
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/How-to-Select-a-Model-For-Your-Time-Series-Prediction-Task_17.png?resize=831%2C122&ssl=1)


ADF 검정의 귀무가설은 단위근이 시계열에 존재한다입니다. 대립가설은 데이터가 고정적이라는 것입니다. 

두번째 값인 p-value를 통해서 정상성 확인이 가능합니다. 0.05보다 작으면 귀무가설을 기각하고 대립가설을 받아들일 수 있습니다. 즉 정상성이 있는 데이터라 볼 수 있다는것 입니다.

### 차분(Differencing)
시계열 데이터에서는 추세나 계절성을 감소 시킬 수 있는데 그게 바로 차분의 역할입니다. 쉽게 이야기하면 정상성을 나타내지 않는 데이터를 정상성을 나태내도록 할 수 있는 요소가 차분이라는 것입니다.

위에서 사용한 데이터에 이 적용해보죠

~~~
prev_co2_value = co2_data.co2.shift()
differenced_co2 = co2_data.co2 - prev_co2_value
differenced_co2.plot()
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/How-to-Select-a-Model-For-Your-Time-Series-Prediction-Task_5.png?ssl=1)

추세가 사라진 모습을 확인해 볼 수 있습니다.

~~~
adf, pval, usedlag, nobs, crit_vals, icbest = adfuller(differenced_co2.dropna())
print( 'ADF 테스트 통계:' , adf)
print( 'ADF p-값:' , pval)
print( '사용된 ADF 지연 수:' , usedlag)
print( 'ADF 관찰 수:' , nobs)
print( 'ADF 임계값:' , crit_vals)
print( 'ADF 최상의 정보 기준:' , icbest)
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/How-to-Select-a-Model-For-Your-Time-Series-Prediction-Task_4.png?resize=851%2C122&ssl=1)

p-value가 0.05보다 작아진 것을 확인 할 수 있습니다. 즉 정상성을 갖는다는 대립 가설이 채택됩니다.


### One-Step vs Multi-Step
일부 모델은 시계열의 다음을 예측하는데 한 번에 여러 단계를 에측하지 못하는 모델들이 존재합니다. 이를 One-Steop 모델이라고 합니다. 여러 단계를 예측이 불가능한 것은 아니지만 결과에 대한 성능은 기대하지 않는게 좋습니다. 예측 값을 사용한 예측은 오류를 급속도로 증가시킬 것입니다.

Multi-Step 모델은 한 번에 여러 단계를 예측하 수 있는 모델입니다. 장기 예측에 적합한 모델입니다. 떄로는 One-Steop 예측을 대신하여 사용할 수 있습니다.


|One-Steop Predict|Multi-Steop Predict|
|-----|-----|
|미래의 한 단계를 위한 예측|미래에 대한 여러 단계 예측 가능|
|예측에 윈도우를 설정한다면 다단계 예측을 할 수 있음|별도의 윈도우 설정이 필요 하지 않음|
|다단계 예측에 대한 성능이 좋지 않음|한 단계 예측보다 다단계 예측에 더 적합|

</br></br>

## 다양한 모델
시계열 데이터 예측을 위한 모델은 크게 3가지로 볼 수 있습니다.

- 고전적인 시계열 모델
- 지도학습 모델
- 딥러닝 기반 모델

3가지에 속하는 모델들이 어떤것들이 있는지 살펴보겠습니다.

